<a href="https://colab.research.google.com/github/janinamue/Tensorflow/blob/main/Text_classification_with_TensorFlow_Hub_Movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification with TensorFlow Hub: Movie reviews

Dieses Notebook **klassifiziert Filmbewertungen anhand des Textes der Bewertung als positiv oder negativ**. Dies ist ein Beispiel für eine **binäre** oder Zwei-Klassen-Klassifizierung, eine wichtige und weit verbreitete Art von maschinellen Lernproblemen.

Das Tutorial demonstriert die grundlegende Anwendung von Transfer Learning mit TensorFlow Hub und Keras.

Es verwendet den **IMDB-Datensatz, der den Text von 50.000 Filmkritiken** aus der Internet Movie Database enthält. Diese werden in **25.000 Rezensionen für das Trainin**g und **25.000 Rezensionen für das Testen** aufgeteilt. Die Trainings- und Testdatensätze sind ausgewogen, d.h. sie enthalten eine gleiche Anzahl positiver und negativer Kritiken.

Dieses Notizbuch verwendet **tf.keras**, eine High-Level-API zum Erstellen und Trainieren von Modellen in TensorFlow, und tensorflow_hub, eine Bibliothek zum Laden trainierter Modelle von TFHub in einer einzigen Codezeile. Eine fortgeschrittene Anleitung zur Textklassifikation mit tf.keras finden Sie im MLCC Text Classification Guide.


In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


*NumPy ist eine Programmbibliothek für die Programmiersprache Python, die eine einfache Handhabung von Vektoren, Matrizen oder generell großen mehrdimensionalen Arrays ermöglicht. Neben den Datenstrukturen bietet NumPy auch effizient implementierte Funktionen für numerische Berechnungen an.*

https://de.wikipedia.org/wiki/NumPy

Laden Sie den IMDB-Datensatz herunter
Der IMDB-Datensatz ist auf imdb reviews oder auf TensorFlow datasets verfügbar. Der folgende Code lädt den IMDB-Datensatz auf Ihre Maschine (oder die Colab-Laufzeitumgebung) herunter:

In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUBO69L/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUBO69L/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Erkunden Sie die Daten
Nehmen wir uns einen Moment Zeit, um das Format der Daten zu verstehen. Jedes Beispiel besteht aus einem Satz, der die Filmkritik darstellt, und einer entsprechenden Bezeichnung. Der Satz ist in keiner Weise vorverarbeitet. Die Bezeichnung ist ein ganzzahliger Wert von entweder 0 oder 1, wobei 0 eine negative Bewertung und 1 eine positive Bewertung bedeutet.

Lassen Sie uns die ersten 10 Beispiele drucken.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

# Build the model

**Das neuronale Netz wird durch das Stapeln von Schichten erstellt, was drei wichtige architektonische Entscheidungen erfordert:**

Wie soll der **Text** dargestellt werden?

Wie viele **Schichten** sollen im Modell verwendet werden?

Wie viele **versteckte Einheite**n sollen für jede Schicht verwendet werden?

In diesem Beispiel bestehen die Eingabedaten aus Sätzen. Die zu 
prognostizierenden Labels sind entweder 0 oder 1.

Eine Möglichkeit, den Text darzustellen, ist die **Umwandlung von Sätzen in Einbettungsvektoren**. Verwenden Sie **eine vortrainierte Texteinbettung als erste Schicht**, was drei Vorteile hat:

Sie müssen sich nicht um die Vorverarbeitung des Textes kümmern,
Sie profitieren vom Transfer-Lernen,
die Einbettung hat eine feste Größe, so dass sie einfacher zu verarbeiten ist.
Für dieses Beispiel verwenden Sie ein vortrainiertes Texteinbettungsmodell von TensorFlow Hub namens **google/nnlm-en-dim50/2**.

Es gibt viele andere vortrainierte Texteinbettungen von TFHub https://tfhub.dev/, die in diesem Tutorial verwendet werden können:

google/nnlm-en-dim128/2 - trainiert mit der gleichen NNLM-Architektur auf den gleichen Daten wie google/nnlm-en-dim50/2, aber mit einer größeren Einbettungsdimension. Größere Einbettungsdimensionen können Ihre Aufgabe verbessern, aber es kann länger dauern, Ihr Modell zu trainieren.
google/nnlm-en-dim128-with-normalization/2 - wie google/nnlm-en-dim128/2, jedoch mit zusätzlicher Textnormalisierung, z. B. durch Entfernen von Satzzeichen. Dies kann hilfreich sein, wenn der Text in Ihrer Aufgabe zusätzliche Zeichen oder Interpunktion enthält.
google/universal-sentence-encoder/4 - ein viel größeres Modell, das 512 dimensionale Einbettungen liefert, die mit einem Deep Averaging Network (DAN)-Encoder trainiert wurden.
Und viele mehr! Weitere Modelle zur Texteinbettung finden Sie auf TFHub.

Lassen Sie uns zunächst eine **Keras-Schicht** erstellen, die ein TensorFlow-Hub-Modell zum Einbetten der Sätze verwendet, und es an ein paar Eingabebeispielen ausprobieren. Beachten Sie, dass unabhängig von der Länge des Eingabetextes, die Ausgabeform der Einbettungen ist: (num_examples, embedding_dimension).

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

Die **Schichten** werden nacheinander gestapelt, um den Klassifikator aufzubauen:

Die erste Schicht ist eine TensorFlow Hub-Schicht. Diese Schicht verwendet ein vortrainiertes Saved Model, um einen Satz in seinen Einbettungsvektor abzubilden. Das von Ihnen verwendete vorgefertigte **Texteinbettungsmodell** (google/nnlm-en-dim50/2) zerlegt den Satz in Token, bettet jedes Token ein und kombiniert dann die Einbettung. Die resultierenden Dimensionen sind: (num_examples, embedding_dimension). Für dieses NNLM-Modell beträgt die Einbettungsdimension 50.

Dieser Ausgangsvektor mit fester Länge wird durch eine vollständig verbundene **(dichte) Schicht mit 16** versteckten Einheiten geleitet.

Die letzte Schicht ist dicht mit einem **einzigen Ausgangsknoten** verbunden.
Lassen Sie uns das Modell kompilieren.

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


# Verlustfunktion und Optimierer

Ein Modell benötigt eine Verlustfunktion und einen Optimierer für das Training. Da es sich um ein **binäres Klassifizierungsproblem** handelt und das Modell **Logits** ausgibt (eine Einheitsschicht mit einer linearen Aktivierung), verwenden Sie die Verlustfunktion **binary_crossentropy**.

*Alternativ: Dies ist nicht die einzige Verlustfunktion, Sie könnten z. B. auch mean_squared_error wählen. Aber im Allgemeinen ist binary_crossentropy besser für den Umgang mit Wahrscheinlichkeiten geeignet - sie misst den "Abstand" zwischen Wahrscheinlichkeitsverteilungen, oder in unserem Fall zwischen der Grundwahrscheinlichkeitsverteilung und den Vorhersagen.*

Später, wenn Sie sich mit Regressionsproblemen befassen (z. B. mit der Vorhersage des Preises eines Hauses), werden Sie sehen, wie Sie eine andere Verlustfunktion, den **mittleren quadratischen Fehler**, verwenden können.

Konfigurieren Sie nun das Modell für die Verwendung eines Optimierers und einer Verlustfunktion:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Das Modell trainieren
Trainieren Sie das Modell für 10 Epochen in Mini-Batches von 512 Stichproben. Dies entspricht 10 Iterationen über alle Stichproben in den Tensoren x_train und y_train. Überwachen Sie während des Trainings den Verlust und die Genauigkeit des Modells anhand der 10.000 Stichproben aus dem Validierungssatz:

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 20s 619ms/step - loss: 0.6591 - accuracy: 0.5231 - val_loss: 0.6060 - val_accuracy: 0.5941
Epoch 2/10
30/30 [==============================] - 19s 619ms/step - loss: 0.5437 - accuracy: 0.6771 - val_loss: 0.5069 - val_accuracy: 0.7407
Epoch 3/10
30/30 [==============================] - 19s 626ms/step - loss: 0.4169 - accuracy: 0.8123 - val_loss: 0.4102 - val_accuracy: 0.8210
Epoch 4/10
30/30 [==============================] - 19s 621ms/step - loss: 0.3048 - accuracy: 0.8846 - val_loss: 0.3537 - val_accuracy: 0.8352
Epoch 5/10
30/30 [==============================] - 19s 613ms/step - loss: 0.2275 - accuracy: 0.9188 - val_loss: 0.3238 - val_accuracy: 0.8559
Epoch 6/10
30/30 [==============================] - 19s 618ms/step - loss: 0.1721 - accuracy: 0.9456 - val_loss: 0.3104 - val_accuracy: 0.8618
Epoch 7/10
30/30 [==============================] - 19s 614ms/step - loss: 0.1277 - accuracy: 0.9633 - val_loss: 0.3083 - val_accuracy: 0.8704

#Bewerten Sie das Modell
Schauen wir uns an, wie das Modell abschneidet. Es werden zwei Werte zurückgegeben. Verlust (eine Zahl, die unseren Fehler darstellt, kleinere Werte sind besser) und Genauigkeit.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 4s - loss: 0.3598 - accuracy: 0.8540 - 4s/epoch - 91ms/step
loss: 0.360
accuracy: 0.854


Mit diesem recht naiven Ansatz wird eine Genauigkeit von etwa 87 % erreicht. Mit fortschrittlicheren Ansätzen sollte das Modell näher an 95 % herankommen.

Weitere Lektüre
Eine allgemeinere Methode für die Arbeit mit String-Eingaben und eine detailliertere Analyse des Fortschritts von Genauigkeit und Verlust während des Trainings finden Sie im Tutorial Textklassifikation mit vorverarbeitetem Text.
Probieren Sie weitere textbezogene Tutorien mit trainierten Modellen von TFHub aus.
https://tfhub.dev/ 